In [1]:
#default_exp exception_fns

In [2]:
#export
import os
import numpy as np
import pandas as pd
import plotly
import matplotlib.pyplot as plt
import datetime
import plotly.express as px
from plotly.offline import init_notebook_mode , plot_mpl
import psutil
import plotly.io as pio
from IPython.display import display, HTML
import random
from vibsource.date_exclusions import *

In [3]:
init_notebook_mode(connected = True)
pio.templates.default = "plotly_white"
#from vibsource.exception_fns import *

## INPUT AREA

In [4]:
#hide
data_folder =  ".\data-cd\\Pump6\\"
asset_id = "aid-g-pump-6"
asset_name = 'g-pump-6'
metric_type = "pressure"
direction = None # Placeholder for the dataset column.
lst_exclude_dates = lst_exclude_dates_pump5;

In [5]:
#export
HOUSE_PRESSURE = "HousePressure"
INLET_PRESSURE = "InletPressure"
OUTLET_PRESSURE = "OutletPressure"
#lst_exclude_dates_dec_19 = [(3,12,2019),(4,12,2019),(5,12,2019),(6,12,2019),(7,12,2019),(8,12,2019),(9,12,2019),
#                    (10,12,2019),(11,12,2019),(12,12,2019),(13,12,2019)]
#lst_exclude_dates_jan_20 = [(13,1,2020),(14,1,2020),(15,1,2020),(16,1,2020),(17,1,2020),(18,1,2020),(19,1,2020),
#                    (20,1,2020),(21,1,2020),(22,1,2020),(23,1,2020)]

In [6]:
#export
def dmytostr(d,m,y):
    return str(d) + "-" + str(m) + "-" + str(y)

def dmytotuple(d,m,y):
    return (d,m,y)

def get_common_dates_list(df1, df2, df3):
    df = pd.merge(df1, df2, how='inner')
    df_common_dates_exception_data = pd.merge(df, df3, how='inner')
    df_common_dates_exception_data['datestr'] = list(map(dmytostr, df_common_dates_exception_data['d'], 
                                               df_common_dates_exception_data['month'],
                                               df_common_dates_exception_data['year']))
    lst_common_dates = list(map(dmytotuple, df_common_dates_exception_data['d'], 
                                               df_common_dates_exception_data['month'],
                                               df_common_dates_exception_data['year']))
    return lst_common_dates

def add_date_components(df, date_col_name = 'Date_Hour'):
    # ensure that the column provided is datetime.
    if (df[date_col_name].dtype != 'datetime64[ns]'):
        df[date_col_name] = pd.to_datetime(df[date_col_name])
    df = df.assign(d=df[date_col_name].dt.day)
    df = df.assign(dayofweek=df[date_col_name].dt.dayofweek)
    df = df.assign(year=df[date_col_name].dt.year)
    df = df.assign(month=df[date_col_name].dt.month)
    df = df.assign(hour=df[date_col_name].dt.hour)
    df = df.assign(minute=df[date_col_name].dt.minute)
    df = df.assign(second=df[date_col_name].dt.second)
    df = df.assign(week=df[date_col_name].dt.week)
    return(df)


def remove_rows_from_df(df, lst_exclude_dates):
    #lst_exclude_dates = [(2,3,2019), (2,4,2019)]
    for d,m,y in lst_exclude_dates:
        df = df[~((df.d == d)&(df.month==m)&(df.year==y))]
    return df

def get_one_day_data(df , day, month, year):
    df_one_day_data = df[(df['d'] == day) & (df['month'] == month) & (df['year'] == year) ]
    return df_one_day_data

def get_dmy_from_dates(df, i):
    assert df.shape[0] > i-1, "DataAnalysis_DataRangeError: Date range out of bounds, max you can use is:" + str(df.shape[0])
    y = df.iloc[i]['year']
    d = df.iloc[i]['d']
    m = df.iloc[i]['month']
    return d,m, y


def get_days_trace(df, d , m, y, x_column_name, y_column_name, trace_name = "default"):
    cond_match = ((df['d'] == d) & (df['month'] == m)  & (df['year'] == y))
    df = df[cond_match].reset_index()
    #df = df.query('date == 2009 & id == 35988')
    _line = plotly.graph_objs.Scatter(x=list(df[x_column_name]),
                                      y =df[y_column_name], name = trace_name,
                                    mode='markers')
    return _line
    #return get_plotly_trace(df, x_column_name, y_column_name, trace_name)

In [90]:
#export
class AssetInfo:
    HOUSE_PRESSURE = "HousePressure"
    INLET_PRESSURE = "InletPressure"
    OUTLET_PRESSURE = "OutletPressure"
    
    
    def __init__(self, data_folder, asset_id, metric_type
                 ,direction = None, asset_name = "Not provided"):
        print("init")
        self.data_folder = data_folder
        self.asset_id = asset_id
        self.metric_type = metric_type
        self.direction = direction
        self.asset_name = asset_name
        self.image_dir = "./images/" + asset_id + "/"
        print(self.image_dir)
        self.A_overall_plot_dir = self.image_dir + "A-overall-data-plots/"
        self.B_common_exception_days_dir  = self.image_dir + "B-Common-Exception-Days/"
        self.C_unique_exception_days_dir  = self.image_dir + "C-Unique-Exception-Days/"
        self.D_normal_days_dir  = self.image_dir + "D-Normal-Days/"
        self.E_monthly_plots_dir  = self.image_dir + "E-Monthly-Plots/"
        
        self.result_file = "./" + "summary-" + asset_id + ".txt"
        if not (os.path.exists(self.image_dir)):
            print("creating", self.image_dir)
            os.mkdir(self.image_dir) 
 
        if not (os.path.exists(self.D_normal_days_dir)):
            os.mkdir(self.D_normal_days_dir)  
            #print("creating", self.D_normal_days_dir)
        if not (os.path.exists(self.E_monthly_plots_dir)):
            os.mkdir(self.E_monthly_plots_dir)
        if not (os.path.exists(self.C_unique_exception_days_dir)):
            os.mkdir(self.C_unique_exception_days_dir)  
        if not (os.path.exists(self.B_common_exception_days_dir)):
            os.mkdir(self.B_common_exception_days_dir)  
        if not (os.path.exists(self.A_overall_plot_dir)):
            os.mkdir(self.A_overall_plot_dir)     
            
        
        #self.variable_name = variable_name
    def write_data(self , strtext):
        #filename_debug = dir_name_debug +  "00debug_sentences.txt"
        with open(self.result_file, 'a') as the_file:
            the_file.write( strtext +  "\n")
        the_file.close()
    def print_line(self):
        self.write_data("------------------------------------------------------------------------------------------------------")
    def get_file_path(self, variable_name):
         return self.data_folder + variable_name + ".csv"
        
    def get_raw_data(self, variable_name):
        df = pd.read_csv(self.get_file_path(variable_name))
        df = add_date_components(df, "DateString")
        if ( variable_name == HOUSE_PRESSURE):
            self.df_housepressure_raw = df
        if ( variable_name == INLET_PRESSURE):
            self.df_inletpressure_raw = df
        if ( variable_name == OUTLET_PRESSURE):
            self.df_outletpressure_raw = df
        return df
    
    def get_num_days_for_all_data(self, variable_name = OUTLET_PRESSURE):
        """
        getting dates when negative data is seen.
        """
        df = pd.read_csv(self.get_file_path(variable_name))
        df_raw_negative_data = add_date_components(df, "DateString")
        df_dates_neg_data = df_raw_negative_data[['year', 'month', 'd']].drop_duplicates()
        #df_dates_neg_data.shape
        return(df_dates_neg_data.shape[0])
    
    def get_dates_for_negative_data(self, variable_name = OUTLET_PRESSURE, thresh = 0):
        """
        getting dates when negative data is seen.
        """
        df = pd.read_csv(self.get_file_path(variable_name))
        df_raw_negative_data = add_date_components(df, "DateString")
        df_raw_negative_data = df_raw_negative_data[df_raw_negative_data['Value'] < thresh]
        #print(df_raw_negative_data.head())
        df_dates_neg_data = df_raw_negative_data[['year', 'month', 'd']].drop_duplicates()
        #df_dates_neg_data.shape
        return(df_dates_neg_data)
    def plot_single_variable_uncommon_day(self, day, month, year, variable_name = HOUSE_PRESSURE
                        , title = "<please provide>", printplot = False):
        df_raw_data = pd.read_csv(self.get_file_path(variable_name))
        df_raw_data = add_date_components(df_raw_data, "DateString")
        #title = " {} on {} {} {} for asset {}-Uncommon Day Ex.".format(variable_name, day, month, year, self.asset_id)
        title = "{}-{}-{}-variable-{}-asset-{}-c-unique_days".format(year,f'{month:02}',f'{day:02}', variable_name, self.asset_id)
        #print(title)
        df = get_one_day_data(df_raw_data, day, month, year)
        file_name = self.C_unique_exception_days_dir + title + ".png"
        #file_name = self.C_unique_exception_days_dir + "C-Uncommon-day-" 
        #+ str(day) + "-" + str(month) + "-" + str(year) + "-" + self.asset_name + "-" + variable_name + ".png"
        #print(file_name)
        if printplot:
            fig = px.scatter(df, x="DateString", y="AvgValue", title=title)
            #fig.write_image(file_name)
            fig.show()
        # MATPLOTLIB SAVE: Using matplotlib to save data since plotly takes lot of time
        _ = plt.plot( 'DateString', 'AvgValue', data=df, marker='', linewidth=2, color='blue', label = variable_name)
        #_ = plt.title(title)
        _ = plt.title( title, fontdict = {'fontsize' : 6})
        _ = plt.legend()
        plt.xticks(fontsize=6, rotation=60)
        _ = plt.savefig( file_name, dpi=800)
        plt.close()
        
    def plot_single_variable_normal_day(self, day, month, year, variable_name = HOUSE_PRESSURE
                        , title = "<please provide>", printplot = False):
        df_raw_data = pd.read_csv(self.get_file_path(variable_name))
        df_raw_data = add_date_components(df_raw_data, "DateString")
        #title = " {} on {} {} {} for asset {} - Normal Day".format(variable_name, day, month, year, self.asset_id)
        title = "{}-{}-{}-variable-{}-asset-{}-d-normal-days".format(year,f'{month:02}',f'{day:02}', variable_name, self.asset_id)
        #print(title)
        df = get_one_day_data(df_raw_data, day, month, year)
        #file_name = self.D_normal_days_dir + "D-Normal-day-" + str(day) + "-" + str(month) + "-" + str(year) + self.asset_name + "-" + variable_name + ".png"
        file_name = self.D_normal_days_dir + title + ".png"
        
        #print(file_name)
        if printplot:
            fig = px.scatter(df, x="DateString", y="AvgValue", title=title)
            #fig.write_image(file_name)
            fig.show()
        # Using matplotlib to save data since plotly takes lot of time
        _ = plt.plot( 'DateString', 'AvgValue', data=df, marker='', linewidth=2, color='blue', label = variable_name)
        #_ = plt.title(title)
        _ = plt.title( title, fontdict = {'fontsize' : 6})
        _ = plt.legend()
        plt.xticks(fontsize=6, rotation=60)
        _ = plt.savefig( file_name, dpi=800)
        plt.close()
        #fig.show()
    def plot_overall_data(self, df, variable_name):
        title = " Overall data for variable: {}, for asset: {}".format(variable_name, self.asset_id)
        #fig = px.scatter(df, x="DateString", y="AvgValue", title=title)
        #_ = plt.plot( 'DateString', 'AvgValue', data=df, marker='', linewidth=2, color='blue', label = variable_name)
        _ = plt.scatter(df["DateString"], df["AvgValue"], marker = ',', s = .5)
        plt.xticks(fontsize=6, rotation=60)
        #_ = plt.title(title)
        _ = plt.title( title, fontdict = {'fontsize' : 6})
        #_ = plt.legend()
        file_name = self.A_overall_plot_dir + "A-overall-data" + self.asset_name + "-" + variable_name + ".png"
        _ = plt.savefig( file_name, dpi=800)
        plt.close()
        #fig.write_image(file_name)
    def save_plot_for_all_vars_for_day(self, d, m, y, df_ip, df_hp, df_op, show_on_page = False):
        #d = 20
        #m = 2
        #y = 2019
        print(d,m,y)
        cond_match_hp = ((df_hp['d'] == d) & (df_hp['month'] == m)  & (df_hp['year'] == y))
        cond_match_ip = ((df_ip['d'] == d) & (df_ip['month'] == m)  & (df_ip['year'] == y))
        cond_match_op = ((df_op['d'] == d) & (df_op['month'] == m)  & (df_op['year'] == y))
        df_hpd = df_hp[cond_match_hp].reset_index()
        df_ipd = df_ip[cond_match_ip].reset_index()
        df_opd = df_op[cond_match_op].reset_index()
        #title = "B-Common-Day-{} on {} {} {} for asset {}".format('all-variables', d, m, y, self.asset_id)
        title = "{}-{}-{}-asset-{}-b-common-exception-days".format(y,f'{m:02}',f'{d:02}', self.asset_id)
        _ = plt.plot( 'DateString', 'AvgValue', data=df_hpd, marker='', linewidth=2, color='blue', label = "house")
        _ = plt.plot( 'DateString', 'AvgValue', data=df_opd, marker='', color='olive', linewidth=2, label = "outlet")
        _ = plt.plot( 'DateString', 'AvgValue', data=df_ipd, marker='', color='red', linewidth=2, label = "inlet")
        #title = "Comparison of all vars on {} {} {}".format(d, m, y)
        plt.xticks(fontsize=6, rotation=60)
        #_ = plt.title(title)
        _ = plt.title( title, fontdict = {'fontsize' : 6})
        _ = plt.legend()
        _ = plt.savefig( self.B_common_exception_days_dir  + title + ".png", dpi=800)
        plt.close()
        if show_on_page:
            self.show_plotly_plot_for_all_vars_for_day(d, m, y, df_ip, df_hp, df_op)
        #plot_mpl(fig, image='png')
        #fig.show()
    def show_plotly_plot_for_all_vars_for_day(self, d, m, y, df_ipd, df_hpd, df_opd):
        hp_trace = get_days_trace(df_hp, d, m, y, 'DateString', 'AvgValue', self.HOUSE_PRESSURE )
        ip_trace = get_days_trace(df_ip, d, m, y, 'DateString', 'AvgValue', self.INLET_PRESSURE )
        op_trace = get_days_trace(df_op, d, m, y, 'DateString', 'AvgValue', self.OUTLET_PRESSURE )
        #plotly.offline.iplot([hp_trace, ip_trace, op_trace], filename = './test', image='png')
        plotly.offline.iplot([hp_trace, ip_trace, op_trace])
    def save_plot_for_all_vars_for_a_month( self, m, y, df_ip, df_hp, df_op, show_on_page = False):
        #d = 20
        #m = 2
        #y = 2019
        #print(d,m,y)
        cond_match_hp = ((df_hp['month'] == m)  & (df_hp['year'] == y))
        cond_match_ip = ((df_ip['month'] == m)  & (df_ip['year'] == y))
        cond_match_op = ((df_op['month'] == m)  & (df_op['year'] == y))
        df_hpd = df_hp[cond_match_hp].reset_index()
        df_ipd = df_ip[cond_match_ip].reset_index()
        df_opd = df_op[cond_match_op].reset_index()
        #title = "E-monthly-{} for {} {} for asset {}".format('all-variables', m, y, self.asset_id)
        title = "{}-{}-all-variables-asset-{}-e-monthly-data".format(y,f'{m:2}', self.asset_id)
        _ = plt.plot( 'DateString', 'AvgValue', data=df_hpd, marker='.', linewidth=2, color='blue', label = "house")
        _ = plt.plot( 'DateString', 'AvgValue', data=df_opd, marker='.', color='olive', linewidth=2, label = "outlet")
        _ = plt.plot( 'DateString', 'AvgValue', data=df_ipd, marker='.', color='red', linewidth=2, label = "inlet")
        plt.xticks(fontsize=6, rotation=60)
        #_ = plt.title(title)
        _ = plt.title( title, fontdict = {'fontsize' : 6})
        _ = plt.legend()
        _ = plt.savefig( self.E_monthly_plots_dir  + title + ".png", dpi=800)
        plt.close()

In [91]:
#plt.scatter?

In [92]:
obj_asset_info = AssetInfo(data_folder, asset_id,  metric_type, direction, asset_name)

init
./images/aid-g-pump-6/


In [64]:
df_op = obj_asset_info.get_raw_data(OUTLET_PRESSURE)
df_hp = obj_asset_info.get_raw_data(HOUSE_PRESSURE)
df_ip = obj_asset_info.get_raw_data(INLET_PRESSURE)

In [65]:
df_op.shape[1]

16

In [66]:
obj_asset_info.write_data("Generating summary for asset: {} : {}\n".format(asset_id, asset_name))

In [67]:
obj_asset_info.write_data("Number of observations for Outlet Pressue: {}".format(df_op.shape[0]))
obj_asset_info.write_data("Number of observations for Inlet Pressue: {}".format(df_ip.shape[0]))
obj_asset_info.write_data("Number of observations for Housing Pressue: {}".format(df_hp.shape[0]))

In [68]:
df_op = remove_rows_from_df(df_op, lst_exclude_dates); print(df_op.shape)
df_ip = remove_rows_from_df(df_ip, lst_exclude_dates); print(df_ip.shape)
df_hp = remove_rows_from_df(df_hp, lst_exclude_dates); print(df_hp.shape)

(10669, 16)
(9837, 16)
(9837, 16)


In [69]:
obj_asset_info.write_data("Number of observations for Outlet Pressue after Exclusion: {}".format(df_op.shape[0]))
obj_asset_info.write_data("Number of observations for Inlet Pressue after Exclusion: {}".format(df_ip.shape[0]))
obj_asset_info.write_data("Number of observations for Housing Pressue after Exclusion: {}".format(df_hp.shape[0]))

In [70]:
print(df_op.shape);print(df_hp.shape);print(df_ip.shape);

(10669, 16)
(9837, 16)
(9837, 16)


## Plot Overall Data - whole timeframe

In [93]:
obj_asset_info.plot_overall_data(df_op, OUTLET_PRESSURE)

In [88]:
obj_asset_info.plot_overall_data(df_ip, INLET_PRESSURE)

In [89]:
obj_asset_info.plot_overall_data(df_hp, HOUSE_PRESSURE)

## Number of Days for Data

In [19]:
num_days_op = obj_asset_info.get_num_days_for_all_data(variable_name = OUTLET_PRESSURE)
num_days_ip = obj_asset_info.get_num_days_for_all_data(variable_name = INLET_PRESSURE)
num_days_hp = obj_asset_info.get_num_days_for_all_data(variable_name = HOUSE_PRESSURE)

In [20]:
obj_asset_info.write_data("Number of days for Outlet Pressue overall: {}".format(num_days_op))
obj_asset_info.write_data("Number of days for Inlet Pressue with overall: {}".format(num_days_ip))
obj_asset_info.write_data("Number of days for Housing Pressue with overall: {}".format(num_days_hp))

## Getting exception data and removing them

In [21]:
df_negative_data_dates_hp = obj_asset_info.get_dates_for_negative_data(HOUSE_PRESSURE, thresh =1)
df_negative_data_dates_ip = obj_asset_info.get_dates_for_negative_data(INLET_PRESSURE, thresh = 1)
df_negative_data_dates_op = obj_asset_info.get_dates_for_negative_data(OUTLET_PRESSURE, thresh = 1)

df_negative_data_dates_hp = remove_rows_from_df(df_negative_data_dates_hp, lst_exclude_dates)
df_negative_data_dates_ip = remove_rows_from_df(df_negative_data_dates_ip, lst_exclude_dates)
df_negative_data_dates_op = remove_rows_from_df(df_negative_data_dates_op, lst_exclude_dates)

In [22]:
obj_asset_info.write_data("Number of days for Outlet Pressue with exceptional data: {}".format(df_negative_data_dates_op.shape[0]))
obj_asset_info.write_data("Number of days for Inlet Pressue with exceptional data: {}".format(df_negative_data_dates_ip.shape[0]))
obj_asset_info.write_data("Number of days for Housing Pressue with exceptional data: {}".format(df_negative_data_dates_hp.shape[0]))

## Getting dates when exceptional data is found across all variables (Common)

In [23]:
#def get_normal_dates(df, df_exception_days):

In [24]:
lst_common_dates = get_common_dates_list(df_negative_data_dates_hp,
                                        df_negative_data_dates_ip,
                                        df_negative_data_dates_op)
#len(lst_common_dates)

In [25]:
obj_asset_info.write_data("Days when exceptional data is found across all variables: {}".format(len(lst_common_dates)))

### Common dates spanning all three variable with exceptional data

In [26]:
def get_normal_dates(df, df_exception_days):
    #df = df_hp
    #df_exception_days = df_negative_data_dates_hp
    lst_exception_dates = list(map(dmytotuple, df_exception_days['d'], 
                                               df_exception_days['month'],
                                               df_exception_days['year']))
    df = remove_rows_from_df(df, lst_exception_dates)
    lst_normal_dates = list(map(dmytotuple, df['d'], 
                                               df['month'],
                                               df['year']))
    lst_normal_dates = list(set(lst_normal_dates))
    return lst_normal_dates

In [27]:
def save_normal_plots(df, variable_name, number_of_instances, df_exception_dates):
    listtoprint = get_normal_dates(df, df_exception_dates)
    list_index = get_random_list(listtoprint, number_of_instances)
    counter = 0
    for l in list_index:
        #print(listtoprint[l])
        counter += 1
        print(counter)
        obj_asset_info.plot_single_variable_normal_day(*listtoprint[l], variable_name)
    # Set the variable to True to print on page , but it slows down the page.
        #obj_asset_info.plot_single_variable_normal_day( *l, variable_name = variable_name)

In [28]:
def save_exceptional_plots_for_common_days(number_of_instances, lst_common_dates, df_hp, df_ip, df_op):
    #listtoprint = get_normal_dates(df, df_exception_days)
    list_index = get_random_list(lst_common_dates, number_of_instances)
    counter = 0
    #print(list_index)
    for l in list_index:
        counter += 1
        print(counter)
        obj_asset_info.save_plot_for_all_vars_for_day(*lst_common_dates[l],df_hp=df_hp, df_ip= df_ip, df_op = df_op)
        #save_plot_for_all_vars_for_a_month

In [29]:
def save_uncommon_exception_plots(df, variable_name, number_of_instances, lst_common_dates):
    lst_exception_uncommon_dates = get_uncommon_dates(df, lst_common_dates)
    #print(len(lst_exception_uncommon_dates))
    list_index = get_random_list(lst_exception_uncommon_dates, number_of_instances)
    counter = 0
    #print("li", list_index)
    for l in list_index:
        #print(lst_exception_uncommon_dates[l])
        # Set the variable to True to print on page , but it slows down the page.
        counter += 1
        print(counter)
        obj_asset_info.plot_single_variable_uncommon_day( *lst_exception_uncommon_dates[l], variable_name = variable_name)
    return lst_exception_uncommon_dates

In [30]:
def get_uncommon_dates(df, lst_common_dates):
    df_negative_data_uncommon = remove_rows_from_df(df, lst_common_dates)
    lst_negative_data_uncommon = list(map(dmytotuple, df_negative_data_uncommon['d'], 
                                               df_negative_data_uncommon['month'],
                                               df_negative_data_uncommon['year']))
    #print(lst_negative_data_uncommon)
    return lst_negative_data_uncommon

In [31]:
def get_random_list(lst_dates, num_to_show=5):
    #print(num_to_show,len(lst_dates))
    #print("rs", random.sample(range(0, len(lst_dates) - 1), len(lst_dates) - 1))
    if ( num_to_show < len(lst_dates)):
        randomlist = random.sample(range(0, len(lst_dates) - 1), num_to_show)
    if (num_to_show >= len(lst_dates)):
        randomlist = list(range(0, len(lst_dates)))
    # fixing when it has only one element.
    if (len(lst_dates) == 1):
        randomlist = [0]
    return randomlist

In [32]:
#", ".join([str(x) for x in lst_common_dates])

## Comparing all three variables on some of the exceptional dates

In [33]:
save_exceptional_plots_for_common_days(999, lst_common_dates, df_hp, df_ip, df_op)

## Plotting for Uncommon Dates

### Plotting for uncommon dates - Housepressure

In [34]:
lst_uncommon_dates_hp = save_uncommon_exception_plots(df_negative_data_dates_hp, HOUSE_PRESSURE, 999, lst_common_dates)
obj_asset_info.write_data("Days when exceptional data is found for housepressure (not others): {}".format(len(lst_uncommon_dates_hp)))

In [35]:
print(len(lst_uncommon_dates_hp))

0


### Plotting for uncommon dates - Outlet pressure

In [36]:
lst_uncommon_dates_op = save_uncommon_exception_plots(df_negative_data_dates_op, OUTLET_PRESSURE, 999, lst_common_dates)
obj_asset_info.write_data("Days when exceptional data is found for outlet pressure (not others): {}".format(len(lst_uncommon_dates_op)))

In [37]:
#lst_uncommon_dates_op

### Plotting for uncommon dates - Inlet pressure

In [38]:
lst_uncommon_dates_ip = save_uncommon_exception_plots(df_negative_data_dates_ip, INLET_PRESSURE, 999, lst_common_dates)
obj_asset_info.write_data("Days when exceptional data is found for inletpressure (not others): {}".format(len(lst_uncommon_dates_ip)))

## Plotting Normal Days behaviour

### Inlet pressure on Normal Day

In [39]:
save_normal_plots(df_ip, INLET_PRESSURE, 20, df_negative_data_dates_ip)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


### Outlet pressure on Normal Day

In [40]:
save_normal_plots(df_op, OUTLET_PRESSURE, 20, df_negative_data_dates_op)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


### House pressure on Normal Day

In [41]:
save_normal_plots(df_hp, HOUSE_PRESSURE, 20, df_negative_data_dates_hp)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


## Logging

In [42]:
### Logging when Uncommon Dates are found


In [43]:
obj_asset_info.print_line()

In [44]:
obj_asset_info.write_data("Dates when exceptional data is found for house pressure (not others): {}".format((lst_uncommon_dates_hp)))

In [45]:
obj_asset_info.print_line()

In [46]:
obj_asset_info.write_data("Dates when exceptional data is found for inlet pressure (not others): {}".format((lst_uncommon_dates_ip)))

In [47]:
obj_asset_info.print_line()

In [48]:
obj_asset_info.write_data("Dates when exceptional data is found for outlet pressure (not others): {}".format((lst_uncommon_dates_op)))

In [49]:
obj_asset_info.print_line()

In [50]:
obj_asset_info.write_data("Common Dates for exceptional data: {}".format((lst_common_dates)))

In [51]:
obj_asset_info.print_line()

In [52]:
obj_asset_info.write_data("Excluded Dates from Analysis: {}\n".format(lst_exclude_dates))

In [53]:
obj_asset_info.print_line()

In [54]:
#hide
#from nbdev.export import notebook2script
#notebook2script()

In [55]:
obj_asset_info.write_data("End of Summary")

## Monthly Charts Printing

In [56]:
def save_montly_plots_for_all_vars( df_hp, df_ip, df_op):
    df_op_monthly = df_op[['year', 'month']].drop_duplicates(); df_op_monthly
    lst_months = list(map(lambda m, y: (m,y) , df_op_monthly['month'], df_op_monthly['year']))
    for l in lst_months:
        obj_asset_info.save_plot_for_all_vars_for_a_month(*l,df_hp=df_hp, df_ip= df_ip, df_op = df_op)

In [57]:
save_montly_plots_for_all_vars(df_hp, df_ip, df_op)

In [58]:
#plt.plot??

In [59]:
plt.scatter?